In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import cv2
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf;
from tqdm import tqdm
import time

In [ ]:
path = os.getcwd()
path

In [ ]:
# To speed up the notebook, set this value to True to skip intermediate demo steps and only run full pipeline at the end of this notebook
skip_intermediate_demo = True

## Load data

LFW subset locally downloaded

In [ ]:
# Path to the directory containing the subset of LFW dataset
# dataset = 'lfw_10classes_20images'
dataset = 'lfw_30classes_20images'
# dataset = 'lfw_30classes_2images'
# dataset = 'lfw_10images'
# dataset = 'lfw_original' # crashes the kernel, try in Paperspace!
data_path = path + '/data/' + dataset + '/'

In [ ]:
# Load the subset data into a NumPy array
data = []
original_images = []
targets = []
targets = []
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            if os.path.isfile(file_path):
                img = cv2.imread(file_path, cv2.IMREAD_COLOR)
                RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                data.append(RGB_img.flatten())
                original_images.append(RGB_img)
                targets.append(subdir)

data = np.array(data)
original_images = np.array(original_images)

### Explore data

In [ ]:
original_images.shape

In [ ]:
# introspect the images arrays to find the shapes (for plotting)
n, w, h, c = original_images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = data
n_features = X.shape[1]

# the label to predict is the id of the person
target_names = targets
target_names_unique = [target_name for target_name in set(target_names)]
y = LabelEncoder().fit_transform(target_names.copy())
target_names_first_index = [np.where(y == label)[0][0] for label in np.unique(y)]
target_names_first_index.sort()

n_classes = len(target_names_unique)

subdir_counts = {subdir: targets.count(subdir) for subdir in set(targets)}
n_classes_larger_than_5 = len([count for count in subdir_counts.values() if count >= 5])
n_classes_larger_than_10 = len([count for count in subdir_counts.values() if count >= 10])
n_images_from_classes_smaller_than_5 = sum([count for count in subdir_counts.values() if count < 5])
n_images_from_classes_smaller_than_10 = sum([count for count in subdir_counts.values() if count < 10])

In [ ]:
# Print the shape of the data array
print('Data shape:', data.shape)

# Print the list of subdirectories and the number of images per subdirectory
print('Subdirectory counts (unordered):', subdir_counts,'\n')

print(f"There are {n} images in the dataset, {13233 - n} less than the original dataset (of 13233).")
print(f"Each image is {w} pixels wide and {h} pixels tall, with {c} channels (RGB).")
print(f"In total this gives us {n_features} features per image.")
print(f"There are {n_classes} unique targets (classes) in the dataset, of which {n_classes_larger_than_5} have at least five images, and {n_classes_larger_than_10} have at least ten images.")
print(f"There are {n_images_from_classes_smaller_than_5} images from classes with less than five images, and {n_images_from_classes_smaller_than_10} images from classes with less than ten images.")

In [ ]:
original_images.shape

In [ ]:
plot_images = original_images[target_names_first_index[0:10]]
label_array = np.array(target_names)[target_names_first_index[0:10]]

# Create a figure with a 2x5 grid of subplots
fig, axes = plt.subplots(2, 5, figsize=(12, 6))

# Iterate over the subplots and plot each image with its corresponding label
for i, ax in enumerate(axes.flat):
    ax.imshow(plot_images[i])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(str(label_array[i]))

# Show the figure
plt.show()

In [ ]:
original_images.shape

In [ ]:
# the channels: RGB
original_images[0,0,0,:]

In [ ]:
plt.imshow(original_images[0])

In [ ]:
plt.hist(original_images[0,:,:,0])

In [ ]:
def plot_original_faces(indices, max_faces=None, start=0):
    # Get the relevant images from the original_images array
    relevant_images = original_images[indices[start:]]
    
    # If max_faces is set, limit the number of images to plot
    if max_faces is not None:
        relevant_images = relevant_images[:max_faces]
    
    # Determine the number of rows and columns to use in the plot
    num_images = relevant_images.shape[0]
    num_cols = min(num_images, 5)
    num_rows = int(np.ceil(num_images / num_cols))
    
    # Create the plot and add subplots for each image
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
    axs = axs.flatten()
    for i, img in enumerate(relevant_images):
        axs[i].imshow(img)
        axs[i].axis('off')
    
    # Remove any unused subplots
    for j in range(i+1, len(axs)):
        fig.delaxes(axs[j])
    
    plt.tight_layout()
    plt.show()

## Preprocess the images

## Face detection
Based on BlazeFace, taken inspiration from [this repo](https://github.com/ibaiGorordo/BlazeFace-TFLite-Inference). 

In [ ]:
from importlib import reload
from src.blazeface import BlazeFace, BlazeFaceResults
# reload(src.BlazeFace.BlazeFace)
# reload(BlazeFaceResults)

In [ ]:
blazeface_model = BlazeFace(model_type='sparse')
blazeface_model.input_details

In [ ]:
original_test_image = original_images[0]
test_image = blazeface_model.preprocess_input(original_test_image)

In [ ]:
plt.imshow(test_image.numpy().squeeze()) if test_image.shape[0] == 1 else plt.imshow(test_image[0])

#### Extract detections

In [ ]:
face_detection_results = blazeface_model.detect_faces(original_test_image)

In [ ]:
face_detection_results.boxes

In [ ]:
face_detection_results.keypoints

In [ ]:
face_detection_results.scores

In [ ]:
plt.imshow(original_test_image)

In [ ]:
plot_test_image = blazeface_model.draw_detections_single_image(original_test_image, face_detection_results)

In [ ]:
plt.imshow(plot_test_image)

#### Test detections

In [ ]:
if not skip_intermediate_demo:
    blazeface_dense_model = BlazeFace(model_type='dense', score_threshold=0.6)

    face_detection_results = []
    detected_faces_count = 0
    detected_faces_indices = []
    undetected_faces_count = 0
    undetected_faces_indices = []
    double_detected_faces_count = 0
    double_detected_faces_indices = []

    for i in tqdm(range(original_images.shape[0])):
        face_detection_results.append(blazeface_dense_model.detect_faces(original_images[i]))
        if face_detection_results[-1].boxes.shape[0] > 0:
            detected_faces_count += 1
            detected_faces_indices.append(i)
            if face_detection_results[-1].boxes.shape[0] > 1:
                double_detected_faces_count += 1
                double_detected_faces_indices.append(i)

        else:
            undetected_faces_count += 1
            undetected_faces_indices.append(i)

    print(f"For the dense model:")
    print(f"Detected faces: {detected_faces_count} out of {original_images.shape[0]} ({detected_faces_count/original_images.shape[0]*100:.2f}%). Undetected faces: {undetected_faces_count} out of {original_images.shape[0]} ({undetected_faces_count/original_images.shape[0]*100:.2f}%).")
    print(f"Double detected faces: {double_detected_faces_count} out of {original_images.shape[0]} ({double_detected_faces_count/original_images.shape[0]*100:.2f}%).")

In [ ]:
if not skip_intermediate_demo:
    blazeface_model = BlazeFace(model_type='sparse', score_threshold=0.6)

    face_detection_results = []
    detected_faces_count = 0
    detected_faces_indices = []
    undetected_faces_count = 0
    undetected_faces_indices = []
    double_detected_faces_count = 0
    double_detected_faces_indices = []

    for i in tqdm(range(original_images.shape[0])):
        face_detection_results.append(blazeface_model.detect_faces(original_images[i]))
        if face_detection_results[-1].boxes.shape[0] > 0:
            detected_faces_count += 1
            detected_faces_indices.append(i)
            if face_detection_results[-1].boxes.shape[0] > 1:
                double_detected_faces_count += 1
                double_detected_faces_indices.append(i)

        else:
            undetected_faces_count += 1
            undetected_faces_indices.append(i)

    print(f"Detected faces: {detected_faces_count} out of {original_images.shape[0]} ({detected_faces_count/original_images.shape[0]*100:.2f}%). Undetected faces: {undetected_faces_count} out of {original_images.shape[0]} ({undetected_faces_count/original_images.shape[0]*100:.2f}%).")
    print(f"Double detected faces: {double_detected_faces_count} out of {original_images.shape[0]} ({double_detected_faces_count/original_images.shape[0]*100:.2f}%).")

In [ ]:
def plot_face_detections(indices, title, images=original_images, labels=target_names):
    
    plot_images = images[indices]
    label_array = np.array(labels)[indices]
    
    num_images = plot_images.shape[0]

    for i in range(num_images):
        detection_result = blazeface_model.detect_faces(plot_images[i])
        plot_images[i] = blazeface_model.draw_detections_single_image(plot_images[i], detection_result)

    # Calculate the number of rows and columns for the grid
    cols = min(5, num_images)
    rows = int(np.ceil(num_images / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(3*cols, 3*rows))

    # Customize the background color and text color
    bg_color = "#222222"
    text_color = "#ffffff"

    fig.patch.set_facecolor(bg_color)

    for i, ax in enumerate(axes.flat):
        if i < num_images:
            # Plot the image
            if plot_images.shape[-1] == 3:
                ax.imshow(plot_images[i].astype(np.uint8))
            else:
                ax.imshow(plot_images[i], cmap='gray')
            
            # Set the title with the corresponding label
            ax.set_title(f"{label_array[i]}", fontsize=12, color=text_color, y=1.02)
        
        # Remove axis ticks
        ax.set_xticks([])
        ax.set_yticks([])

        # Set the background color for each subplot
        ax.set_facecolor(bg_color)

        # Set the color of the spines (border of the subplots)
        ax.spines['bottom'].set_color(text_color)
        ax.spines['top'].set_color(text_color)
        ax.spines['right'].set_color(text_color)
        ax.spines['left'].set_color(text_color)

    fig.suptitle(title, fontsize=20, y=1.02 + 0.02 * rows, color=text_color)
    plt.tight_layout(rect=[0, 0, 1, 1 - 0.04 * rows])
    plt.show()


In [ ]:
if not skip_intermediate_demo:
    plot_face_detections(target_names_first_index, "Detected faces")

In [ ]:
if not skip_intermediate_demo:
    plot_face_detections(undetected_faces_indices, "Undetected faces")

In [ ]:
if not skip_intermediate_demo:
    plot_face_detections(double_detected_faces_indices, "Double face detections")

## Face alignment

For now I'm simply following [this github](https://sefiks.com/2020/02/23/face-alignment-for-face-recognition-in-python-within-opencv/) script from insightface.

In [ ]:
from src.face_alignment import SimpleFaceAlignment

In [ ]:
test_image = original_images[18]
test_image_face_detection_result = blazeface_model.detect_faces(test_image)
plt.imshow(test_image)

In [ ]:
face_aligner = SimpleFaceAlignment()

In [ ]:
test_aligned_image_results = face_aligner.crop_and_align(test_image, test_image_face_detection_result)
test_aligned_image_results.aligned_faces[0].shape

In [ ]:
plt.imshow(test_aligned_image_results.aligned_faces[0])

In [ ]:
test_aligned_image_results

#### Test alignments

In [ ]:
if not skip_intermediate_demo:
    face_detection_results = []
    face_alignment_results = []

    for idx in tqdm(range(original_images.shape[0])):
        image = original_images[idx]
        face_detection_results.append(blazeface_model.detect_faces(image))
        if (face_detection_results[-1].boxes.shape[0] > 0) & (face_detection_results[-1].boxes.shape[0] < 2):
            face_alignment_results.append(face_aligner.crop_and_align(image, face_detection_results[idx]))
        else:
            face_alignment_results.append(None)
    

In [ ]:
def plot_aligned_faces(aligned_face_results_list, max_faces=None, indices=None, start=0):
    if indices is not None:
        aligned_face_results_list = [aligned_face_results_list[i] for i in indices]

    aligned_faces = [
        face
        for aligned_face_results in aligned_face_results_list
        if aligned_face_results is not None
        for face in aligned_face_results.aligned_faces
    ]

    if start > 0:
        aligned_faces = aligned_faces[start:]

    if max_faces is not None:
        aligned_faces = aligned_faces[:max_faces]

    num_faces = len(aligned_faces)
    cols = min(5, num_faces)
    rows = int(np.ceil(num_faces / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
    fig.suptitle("Aligned Faces", fontsize=20, y=1.02)

    for i, ax in enumerate(axes.flat):
        if i < num_faces:
            if aligned_faces[i].shape[-1] == 3:
                ax.imshow(aligned_faces[i].astype(np.uint8))
            else:
                ax.imshow(aligned_faces[i], cmap='gray')
            ax.set_title(f"Face {i + 1 + start}")

        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()
    plt.show()


In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_alignment_results, max_faces=50)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_alignment_results, max_faces=50, indices=target_names_first_index)

## Face transformation

Follows [this](https://github.com/deepinsight/insightface/blob/be3f7b3e0e635b56d903d845640b048247c41c90/common/face_align.py) example.

In [ ]:
from src.face_alignment import ArcFaceAlignment

In [ ]:
test_image = original_images[25]
plt.imshow(test_image)

In [ ]:
test_image_face_detection_result = blazeface_model.detect_faces(test_image)
if not skip_intermediate_demo:
    test_image_face_detection_result

In [ ]:
test_image.shape

In [ ]:
test_image_landmarks_relative_full = test_image_face_detection_result.keypoints[0][:4]
test_image_landmarks_absolute_full = (test_image_landmarks_relative_full * 250).astype(np.int32)
if not skip_intermediate_demo:
    test_image_landmarks_relative_full

In [ ]:
test_image_face = test_image.copy()
test_image_face = blazeface_model.draw_detections_single_image(test_image_face, test_image_face_detection_result)
plt.imshow(test_image_face)

In [ ]:
crop_face = SimpleFaceAlignment()
test_image_cropped, xmin, ymin = crop_face.crop(test_image, test_image_face_detection_result, 250,250)
test_image_cropped = test_image_cropped[0][:112,:112]
plt.imshow(test_image_cropped)

In [ ]:
arcface_align = ArcFaceAlignment()
test_image_align_result = arcface_align.crop_and_align(test_image, test_image_face_detection_result)
test_image_align_result

In [ ]:
plt.imshow(test_image_align_result.aligned_faces[0])

### Test transformations

In [ ]:
if not skip_intermediate_demo:
    face_detection_results = []
    face_transformation_results = []

    for idx in tqdm(range(original_images.shape[0])):
        image = original_images[idx]
        face_detection_results.append(blazeface_model.detect_faces(image))
        if (face_detection_results[-1].boxes.shape[0] > 0) & (face_detection_results[-1].boxes.shape[0] < 2):
            face_transformation_results.append(arcface_align.crop_and_align(image, face_detection_results[idx]))
        else:
            face_transformation_results.append(None)
        

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_transformation_results, max_faces=30, start=40)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_alignment_results, max_faces=30, start=40)

## Face normalization (optional)

I can use a face mesh model like the one from [MediaPipe](https://github.com/google/mediapipe/blob/master/docs/solutions/face_mesh.md) to normalize faces, as shown [here](https://sefiks.com/2022/05/29/normalization-for-facial-recognition-with-mediapipe/). This probably gives better results, since the bounding boxes and landmarks from BlazeFace are not super accurate.

In [ ]:
from src.face_alignment import FaceMeshAlignment

In [ ]:
face_mesh_model = FaceMeshAlignment(crop=True)

In [ ]:
test_image = original_images[18]
test_image_face_detection_result = blazeface_model.detect_faces(test_image)
test_image_aligned_faces = face_aligner.crop_and_align(test_image, test_image_face_detection_result)

In [ ]:
plt.imshow(test_image)

In [ ]:
plt.imshow(test_image_aligned_faces.aligned_faces[0])

In [ ]:
test_image_normalized_faces = face_mesh_model.normalize_faces(test_image, test_image_face_detection_result)

In [ ]:
plt.imshow(test_image_normalized_faces.aligned_faces[0])

In [ ]:
# run face detection and alignment on all images again
if not skip_intermediate_demo:
    face_detection_results = []
    face_normalization_results = []

    for idx in tqdm(range(original_images.shape[0])):
        image = original_images[idx]
        face_detection_results.append(blazeface_model.detect_faces(image))
        if (face_detection_results[-1].boxes.shape[0] > 0) & (face_detection_results[-1].boxes.shape[0] < 2):
            face_normalization_results.append(face_mesh_model.normalize_faces(image, face_detection_results[idx]))
        else:
            face_normalization_results.append(None)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_normalization_results, max_faces=20)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_alignment_results, max_faces=20)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_normalization_results, max_faces=20, start=20)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_alignment_results, max_faces=20, start=20)

In [ ]:
if not skip_intermediate_demo:
    plot_aligned_faces(face_normalization_results, max_faces=100, start=100)

## Face embeddings

In [ ]:
from src.mobilefacenet import MobileFaceNet

In [ ]:
mobilefacenet_model = MobileFaceNet(model_type="ente_web")
# mobilefacenet_quantized_model = MobileFaceNet(model_type="quantized")

In [ ]:
mobilefacenet_model.interpreter.get_input_details()

In [ ]:
mobilefacenet_model.interpreter.get_output_details()

In [ ]:
# run face detection and alignment/normalization on all images again
face_detection_results = []
face_alignment_results = []
# face_normalization_results = []

for idx in tqdm(range(original_images.shape[0])):
    image = original_images[idx]
    face_detection_results.append(blazeface_model.detect_faces(image))
    if face_detection_results[-1].boxes.shape[0] > 0:
        face_alignment_results.append(arcface_align.crop_and_align(image, face_detection_results[idx]))
        # face_normalization_results.append(face_mesh_model.normalize_faces(image, face_detection_results[idx]))
    else:
        face_alignment_results.append(None)
        # face_normalization_results.append(None)

In [ ]:
# create face embeddings using unquantized MobileFaceNet model
face_embeddings = []
# face_embeddings_normalized = []

for idx in tqdm(range(original_images.shape[0])):
    if (face_alignment_results[idx] is not None): # if face was detected
        if (face_alignment_results[idx].num_faces ==1): # if only one face was detected
            face_embeddings.append(mobilefacenet_model.extract_embedding(face_alignment_results[idx].aligned_faces[0]))
        else:
            face_embeddings.append(None)
    else:
        face_embeddings.append(None)
    # if (face_normalization_results[idx] is not None): # if face was detected
    #     if (face_normalization_results[idx].num_faces ==1): # if only one face was detected
    #         face_embeddings_normalized.append(mobilefacenet_model.extract_embedding(face_normalization_results[idx].aligned_faces[0]))

In [ ]:
len(face_embeddings)

In [ ]:
face_embeddings_squeezed = [array[0] if array is not None else None for array in face_embeddings]

## Indexing the embeddings

### Quick approach with pandas

In [ ]:
import sklearn

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
from collections import Counter

In [ ]:
face_embeddings_used = face_embeddings

# filtered_embeddings = [(idx, arr[0]) for idx, arr in enumerate(face_embeddings) if arr is not None]
filtered_embeddings = [(idx, arr[0]) for idx, arr in enumerate(face_embeddings_used) if arr is not None]

# Separate indices and data into two lists
original_indices, filtered_data_arrays = zip(*filtered_embeddings)
minimum_cluster_size_general = 5
true_labels = y[np.array(original_indices)]
label_counts = Counter(true_labels)
label_map = {label: label if count >= minimum_cluster_size_general else -1 for label, count in label_counts.items()}
true_labels_with_noise = np.vectorize(label_map.get)(true_labels)
# Create a pandas DataFrame from the data arrays
df_embeddings = pd.DataFrame(filtered_data_arrays)

# Add the original index column
df_embeddings['original_index'] = original_indices

# Calculate the cosine distance matrix
temp = time.time()
cosine_dist_matrix = cosine_distances(df_embeddings.drop(columns=['original_index'])).astype(np.float64)
cosine_distance_time = time.time() - temp
temp = time.time()
cosine_dist_matrix_more_precise = pairwise_distances(df_embeddings.drop(columns=['original_index']), metric='cosine').astype(np.float64)
cosine_pairwise_distance_time = time.time() - temp

norm_data = normalize(df_embeddings.drop(columns=['original_index']), norm='l2', axis=1)

In [ ]:
sum(true_labels == 20)

In [ ]:
df_embeddings

### Solid approach with sqlite

## Clustering

### HDBSCAN clustering

In [ ]:
from hdbscan import HDBSCAN

In [ ]:
# Perform HDBSCAN clustering

# # Try out values for the parameters
# minimum_cluster_size = 5
# minimum_samples = 1
# selection_epsilon = 0
# selection_method = 'eom'

# # My best values for the parameters
# minimum_cluster_size = 7
# minimum_samples = 1
# selection_epsilon = 0
# selection_method = 'eom'

# Default values for the parameters
minimum_cluster_size = 5
minimum_samples = 5
selection_epsilon = 0
selection_method = 'eom'

# # Shailesh's best values for the parameters
# minimum_cluster_size = 3
# minimum_samples = 5
# selection_epsilon = 0.6
# selection_method = 'leaf'

# # Standard euclidian distance metric
# clusterer_euclidian = HDBSCAN()
# cluster_labels_euclidian = clusterer_euclidian.fit_predict(df_embeddings.drop(columns=['original_index']))


# Cosine distance metric (precomputed via distance matrix (INEFFICIENT, but best accuracy)))
temp = time.time()
clusterer_precomp = HDBSCAN(metric='precomputed', min_cluster_size=minimum_cluster_size, min_samples=minimum_samples, cluster_selection_epsilon=selection_epsilon, cluster_selection_method=selection_method)
cluster_labels_cosine = clusterer_precomp.fit_predict(cosine_dist_matrix_more_precise)
hdbscan_cosine_time = time.time() - temp
hdbscan_cosine_time += min(cosine_distance_time, cosine_pairwise_distance_time)

# # Angular distance (similar to cosine) via euclidian with normalized data (possibly best balance of accuracy and memory efficiency)
temp = time.time()
clusterer_euclidian = HDBSCAN(min_cluster_size=minimum_cluster_size, min_samples=minimum_samples, cluster_selection_epsilon=selection_epsilon, cluster_selection_method=selection_method) # euclidean distance is the default
cluster_labels_angular = clusterer_euclidian.fit_predict(norm_data)
hdbscan_angular_time = time.time() - temp

# Add the cluster labels as a new column in the DataFrame
df_embeddings['cluster_label'] = cluster_labels_cosine

In [ ]:
df_embeddings

In [ ]:
number_clusters_cosine = len(np.unique(cluster_labels_cosine))-1
ari_cosine = adjusted_rand_score(true_labels_with_noise, cluster_labels_cosine)
ari_cosine_exnoise = adjusted_rand_score(true_labels_with_noise[cluster_labels_cosine != -1], cluster_labels_cosine[cluster_labels_cosine != -1])
ami_cosine = adjusted_mutual_info_score(true_labels_with_noise, cluster_labels_cosine)
ami_cosine_exnoise = adjusted_mutual_info_score(true_labels_with_noise[cluster_labels_cosine != -1], cluster_labels_cosine[cluster_labels_cosine != -1])
percentage_noise_cosine = np.sum(cluster_labels_cosine==-1)/len(cluster_labels_cosine)*100

number_clusters_angular = len(np.unique(cluster_labels_angular))-1
ari_angular = adjusted_rand_score(true_labels_with_noise, cluster_labels_angular)
ari_angular_exnoise = adjusted_rand_score(true_labels_with_noise[cluster_labels_angular != -1], cluster_labels_angular[cluster_labels_angular != -1])
ami_angular = adjusted_mutual_info_score(true_labels_with_noise, cluster_labels_angular)
ami_angular_exnoise = adjusted_mutual_info_score(true_labels_with_noise[cluster_labels_angular != -1], cluster_labels_angular[cluster_labels_angular != -1])
percentage_noise_angular = np.sum(cluster_labels_angular==-1)/len(cluster_labels_angular)*100

In [ ]:
print(f"Results for the cosine distance metric:\n")

print(f"Number of different clusters: {number_clusters_cosine}")
print(f"ARI score for clustering accuracy: {ari_cosine}")
print(f"ARI score excluding noise: {ari_cosine_exnoise}")
print(f"AMI score for clustering accuracy: {ami_cosine}")
print(f"AMI score excluding noise: {ami_cosine_exnoise}")
print(f"Percentage of faces clustered into noise: {percentage_noise_cosine:.2f}%")
print(f"Time needed for clustering: {hdbscan_cosine_time:.2f}s for cosine")

To easily interpret ARI:
The Adjusted Rand Index (ARI) measures the similarity between two clusterings, with 1 indicating a perfect match and 0 implying agreement no better than random chance. A higher ARI value signifies greater similarity between the two clusterings, while a negative value indicates the agreement is worse than random chance.

I think the reason for the low accuracy and high amount of noise is the fact that HDBSCAN works less well on high-dimensional data. The embeddings are in 192 dimensions, while the documentation of HDBSCAN recommends usage for up to 50 a 100 dimensions. 

In [ ]:
print(f"Results for the angular distance metric:\n")

print(f"Number of different clusters: {number_clusters_angular}")
print(f"ARI score for clustering accuracy: {ari_angular}")
print(f"ARI score excluding noise: {ari_angular_exnoise}")
print(f"AMI score for clustering accuracy: {ami_angular}")
print(f"AMI score excluding noise: {ami_angular_exnoise}")
print(f"Percentage of faces clustered into noise: {percentage_noise_angular:.2f}%")
print(f"Time needed for clustering: {hdbscan_angular_time:.2f}s for angular")

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.colors as mcolors

In [ ]:
def plot_tsne_clusters(distance_matrix, labels, figsize=(10, 8), n_components=2, metric='precomputed', init='random', random_state=42):
    # Perform t-SNE on the embeddings
    tsne = TSNE(n_components=n_components, metric=metric, init=init, random_state=random_state)
    embeddings_2d = tsne.fit_transform(distance_matrix)

    # Add the 2D embeddings to a DataFrame
    df_embeddings = pd.DataFrame({'x': embeddings_2d[:, 0], 'y': embeddings_2d[:, 1], 'cluster_label': labels})

    # Define the colormap
    n_clusters = len(np.unique(labels))
    if n_clusters <= 10:
        tab10_cmap = plt.get_cmap('tab10', np.unique(labels).size - 1)
        tab10_cmap_list = tab10_cmap(np.linspace(0, 1, np.unique(labels).size - 1))
        custom_cmap_list = np.vstack(([0.7, 0.7, 0.7, 1.], tab10_cmap_list))
        cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', custom_cmap_list)
    else:
        cmap = plt.get_cmap('nipy_spectral', np.unique(labels).size - 1)

    # Plot the clusters
    plt.figure(figsize=figsize)
    scatter = plt.scatter(df_embeddings['x'], df_embeddings['y'], c=df_embeddings['cluster_label'], cmap=cmap, s=50)
    plt.title("t-SNE visualization of clusters")

    # Add a legend for the cluster colors
    if n_clusters <= 10:
        labels = ["noise"] + [f"Cluster {i}" for i in range(1, n_clusters)]
        handles, _ = scatter.legend_elements()
        legend1 = plt.legend(handles, labels, loc="upper left", title="Clusters")
        plt.gca().add_artist(legend1)
    else:
        # Only show legend for noise
        noise_proxy = plt.scatter([], [], c='grey', s=50)
        plt.legend([noise_proxy], ["noise"], loc="upper left", title="Clusters")

    plt.show()

In [ ]:
plot_tsne_clusters(cosine_dist_matrix, df_embeddings['cluster_label'])

In [ ]:
hdbscan_noise = list(df_embeddings[df_embeddings['cluster_label'] == -1]["original_index"].values)
print(f"Number of faces clustered into noise: {len(hdbscan_noise)} (out of {n_images_from_classes_smaller_than_5}/{n_images_from_classes_smaller_than_10} for min_samples=5/10)")
plot_original_faces(hdbscan_noise, max_faces=10)

### DBSCAN clustering

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances

In [ ]:
# Perform DBSCAN clustering

# test parameters
dbscan_epsilon = 0.3
dbscan_min_samples = 5 

# default parameters
# dbscan_epsilon = 0.5 # 0.5 is default
# dbscan_min_samples = 5 # 5 is default (integer values only)

temp = time.time()
dbscan = DBSCAN(eps=dbscan_epsilon, min_samples=dbscan_min_samples, metric='precomputed')
cluster_labels_dbscan = dbscan.fit_predict(cosine_dist_matrix)
dbscan_time = time.time() - temp

# Add the cluster labels as a new column in the DataFrame
df_embeddings['dbscan_cluster_label'] = cluster_labels_dbscan

In [ ]:
number_clusters_dbscan = len(np.unique(cluster_labels_dbscan))-1
ari_dbscan = adjusted_rand_score(true_labels_with_noise, cluster_labels_dbscan)
ari_dbscan_exnoise = adjusted_rand_score(true_labels_with_noise[cluster_labels_dbscan != -1], cluster_labels_dbscan[cluster_labels_dbscan != -1])
ami_dbscan = adjusted_mutual_info_score(true_labels_with_noise, cluster_labels_dbscan)
ami_dbscan_exnoise = adjusted_mutual_info_score(true_labels_with_noise[cluster_labels_dbscan != -1], cluster_labels_dbscan[cluster_labels_dbscan != -1])
percentage_noise_dbscan = np.sum(cluster_labels_dbscan==-1)/len(cluster_labels_dbscan)*100

In [ ]:
print(f"Results for the DBSCAN clustering:\n")

print(f"Number of different clusters: {number_clusters_dbscan}")
print(f"ARI score for clustering accuracy: {ari_dbscan:.4f}")
print(f"ARI score excluding noise: {ari_dbscan_exnoise:.4f}")
print(f"AMI score for clustering accuracy: {ami_dbscan:.4f}")
print(f"AMI score excluding noise: {ami_dbscan_exnoise:.4f}")
print(f"Percentage of faces clustered into noise: {percentage_noise_dbscan:.2f}%")
print(f"Time taken for DBSCAN clustering: {dbscan_time:.2f}s")

In [ ]:
plot_tsne_clusters(cosine_dist_matrix, df_embeddings['dbscan_cluster_label'])

In [ ]:
dbscan_noise = list(df_embeddings[df_embeddings['dbscan_cluster_label'] == -1]["original_index"].values)
print(f"Number of faces clustered into noise: {len(dbscan_noise)} (out of {n_images_from_classes_smaller_than_5}/{n_images_from_classes_smaller_than_10} for min_samples=5/10)")
plot_original_faces(dbscan_noise, max_faces=10)

### OPTICS

In [ ]:
from sklearn.cluster import OPTICS

In [ ]:
# Perform OPTICS (xi-steep method) clustering

# default parameters
minimum_samples_optics = 2 # 5 is default (integer values only)
minimum_cluster_size_optics = 7 # 0.05 is default (int)
maximum_eps_optics = np.inf # np.inf is default (float or None)
optics_method = 'xi' # 'xi' is default ('xi' or 'dbscan')
optics_dbscan_eps = 0.3 # 0.5 is default (float)
optics_xi = 0.05 # 0.05 is default (float)

temp = time.time()
optics = OPTICS(min_samples = minimum_samples_optics, max_eps= maximum_eps_optics, metric='cosine', cluster_method=optics_method, eps=optics_dbscan_eps, xi=optics_xi, min_cluster_size=minimum_cluster_size_optics)
cluster_labels_optics = optics.fit_predict(norm_data)
optics_time = time.time() - temp

# Add the cluster labels as a new column in the DataFrame
df_embeddings['optics_cluster_label'] = cluster_labels_optics

In [ ]:
number_clusters_optics = len(np.unique(cluster_labels_optics))-1
ari_optics = adjusted_rand_score(true_labels_with_noise, cluster_labels_optics)
ari_optics_exnoise = adjusted_rand_score(true_labels_with_noise[cluster_labels_optics != -1], cluster_labels_optics[cluster_labels_optics != -1])
ami_optics = adjusted_mutual_info_score(true_labels_with_noise, cluster_labels_optics)
ami_optics_exnoise = adjusted_mutual_info_score(true_labels_with_noise[cluster_labels_optics != -1], cluster_labels_optics[cluster_labels_optics != -1])
percentage_noise_optics = np.sum(cluster_labels_optics==-1)/len(cluster_labels_optics)*100

In [ ]:
print(f"Results for the OPTICS clustering:\n")

print(f"Number of different clusters: {number_clusters_optics}")
print(f"ARI score for clustering accuracy: {ari_optics:.4f}")
print(f"ARI score excluding noise: {ari_optics_exnoise:.4f}")
print(f"AMI score for clustering accuracy: {ami_optics:.4f}")
print(f"AMI score excluding noise: {ami_optics_exnoise:.4f}")
print(f"Percentage of faces clustered into noise: {percentage_noise_optics:.2f}%")
print(f"Time taken for OPTICS clustering: {optics_time:.2f}s")

In [ ]:
plot_tsne_clusters(cosine_dist_matrix, df_embeddings['optics_cluster_label'])

In [ ]:
optics_noise = list(df_embeddings[df_embeddings['optics_cluster_label'] == -1]["original_index"].values)
print(f"Number of faces clustered into noise: {len(optics_noise)} (out of {n_images_from_classes_smaller_than_5}/{n_images_from_classes_smaller_than_10} for min_samples=5/10)")
plot_original_faces(optics_noise, max_faces=10)

### HDBSCAN vs DBSCAN

In [ ]:
print('\n' + '-'*80 + '\n')
print("DATA INFORMATION:\n")
# Print the shape of the data array
print(f"Dataset: '{dataset}'")
print('Data shape:', data.shape)

# Print the list of subdirectories and the number of images per subdirectory
print('Subdirectory counts (unordered):', subdir_counts,'\n')

print(f"There are {n} images in the dataset, {13233 - n} less than the original dataset (of 13233).")
print(f"Each image is {w} pixels wide and {h} pixels tall, with {c} channels (RGB).")
print(f"In total this gives us {n_features} features per image.")
print(f"There are {n_classes} unique targets (classes) in the dataset, of which {n_classes_larger_than_5} have at least five images, and {n_classes_larger_than_10} have at least ten images.")
print(f"There are {n_images_from_classes_smaller_than_5} images from classes with less than five images, and {n_images_from_classes_smaller_than_10} images from classes with less than ten images.")


print('\n' + '-'*80 + '\n')


print("CLUSTERING LATENCY:\n")
# print the latency of each clustering algorithm
print("Time taken for each algorithm:")
print(f"HDBSCAN (cosine): {hdbscan_cosine_time:.2f}s")
print(f"HDBSCAN (angular): {hdbscan_angular_time:.2f}s")
print(f"DBSCAN: {dbscan_time:.2f}s")
# print(f"OPTICS: {optics_time:.2f}s")


print('\n' + '-'*80 + '\n')


print("CLUSTERING ACCURACY:\n")

print("Scores for each algorithm:\n")

print("ARI scores:")
print(f"HDBSCAN (cosine): {ari_cosine:.4f}")
print(f"HDBSCAN (angular): {ari_angular:.4f}")
print(f"DBSCAN: {ari_dbscan:.4f}\n")

print("Percentage of faces clustered into noise:")
print(f"HDBSCAN (cosine): {percentage_noise_cosine:.2f}%")
print(f"HDBSCAN (angular): {percentage_noise_angular:.2f}%")
print(f"DBSCAN: {percentage_noise_dbscan:.2f}%\n")

print("ARI scores excluding noise:")
print(f"HDBSCAN (cosine): {ari_cosine_exnoise:.4f}")
print(f"HDBSCAN (angular): {ari_angular_exnoise:.4f}")
print(f"DBSCAN: {ari_dbscan_exnoise:.4f}\n")

print("AMI scores:")
print(f"HDBSCAN (cosine): {ami_cosine:.4f}")
print(f"HDBSCAN (angular): {ami_angular:.4f}")
print(f"DBSCAN: {ami_dbscan:.4f}\n")

print("Percentage of faces clustered into noise:")
print(f"HDBSCAN (cosine): {percentage_noise_cosine:.2f}%")
print(f"HDBSCAN (angular): {percentage_noise_angular:.2f}%")
print(f"DBSCAN: {percentage_noise_dbscan:.2f}%\n")

print("AMI scores excluding noise:")
print(f"HDBSCAN (cosine): {ami_cosine_exnoise:.4f}")
print(f"HDBSCAN (angular): {ami_angular_exnoise:.4f}")
print(f"DBSCAN: {ami_dbscan_exnoise:.4f}\n")

print(f"Number of different clusters: ({n_classes_larger_than_5}/{n_classes_larger_than_10} for min_samples=5/10)")
print(f"HDBSCAN (cosine): {number_clusters_cosine}")
print(f"HDBSCAN (angular): {number_clusters_angular}")
print(f"DBSCAN: {number_clusters_dbscan}\n")